In [1]:
import MDAnalysis as mda
import numpy as np
from MDAnalysis import transformations
from src.lib import run, get_proton_pos
CELL = np.array([19.728, 19.728, 19.728, 90, 90, 90])

In [2]:
def calc_traj_occupancy(traj, proton_trajs):
    traj_occupancy = np.full((len(proton_trajs), len(traj.trajectory)), False)
    for i, pt in enumerate(proton_trajs):
        traj_occupancy[i, pt.iframe] = True
    return traj_occupancy

def calc_displacement(proton_pos):
    cell = np.array(CELL[:3])
    diff = proton_pos[1:] - proton_pos[:-1]
    diff -= np.round(diff / cell) * cell
    pos = [diff[0]]
    for i in range(1, len(diff)):
        pos.append(pos[-1] + diff[i])
    displacement = np.array([np.linalg.norm(pos[i] - pos[0]) ** 2 for i in range(len(pos))])

    return diff, displacement

In [3]:
top_file = 'data/top.xyz'
traj_file = 'data/traj.xtc'

In [4]:
traj = mda.Universe(top_file, traj_file)
transform = transformations.set_dimensions(CELL)
traj.trajectory.add_transformations(transform)
O_idx = traj.select_atoms('name O').indices
H_idx = traj.select_atoms('name H').indices
traj.dimensions = CELL

In [5]:
proton_trajs = run(traj, O_idx, H_idx)

Output()

/home/qxu/Programs/miniconda3/envs/py/lib/python3.12/site-packages/MDAnalysis/core/groups.py:445: 
DeprecationWarning: The 'pbc' kwarg has been deprecated and will be removed in version 3.0., please use 'wrap' 
instead
  warnings.warn("The 'pbc' kwarg has been deprecated and will be "

In [6]:
long_trajs = np.array([t for t in proton_trajs if len(t) > 20])
traj_occupancy = calc_traj_occupancy(traj, long_trajs)
proton_pos = get_proton_pos(traj, traj_occupancy, long_trajs, CELL)

In [7]:
proton_pos

array([[17.51000023, 14.87000084, 15.07999992],
       [19.11800003, 14.37000084, 14.97000122],
       [19.19799995, 14.35000038, 14.96000099],
       ...,
       [12.86000061, 13.93000031, 15.29000092],
       [12.80000114, 13.94000053, 15.31000042],
       [13.03999996, 14.06000042, 17.22000122]])